In [50]:
import numpy as np
import random
import pandas as pd
import lightgbm as lgb
import optuna
from sklearn.metrics import classification_report, balanced_accuracy_score
import json
import os
from optuna.visualization import plot_optimization_history
import matplotlib.pyplot as plt
import time

In [51]:
SEED = 1234
 # Se puede cambiar a "precision" o "recall" o "f1-score"
SCORE = "f1-score"
model_name = 'LightGBM'

In [52]:
def set_seeds(seed):
    random.seed(seed)
    np.random.seed(seed)

Score Trend Changes Score

In [53]:
def get_trend_changes_report_dict(y_test: np.array, y_pred: np.array) -> float:
    """
    Calculate the trend changes score based on the test and predicted values.
    
    Args:
        y_test (np.array): True labels.
        y_pred (np.array): Predicted labels.
        
    Returns:
        float: The trend changes score.
    """
    y_df = pd.DataFrame([y_test, y_pred]).T
    y_df.columns = ["y_test", "y_pred"]
    y_df["y_test_shifted"] = y_df["y_test"].shift(-1)
    y_df["is_changed_trend_test"] = y_df["y_test"] != y_df["y_test_shifted"]
    y_df["y_predict_shifted"] = y_df["y_pred"].shift(-1)
    y_df["is_changed_trend_predict"] = y_df["y_pred"] != y_df["y_predict_shifted"]
    return classification_report(
        y_df["is_changed_trend_test"][:-1], 
        y_df["is_changed_trend_predict"][:-1], 
        digits=4,
        output_dict=True,
        zero_division=0
    )

def trend_changes_score(y_test: np.array, y_pred: np.array) -> float:
    """
    Calculate the trend changes score based on the test and predicted values.
    
    Args:
        y_test (np.array): True labels.
        y_pred (np.array): Predicted labels.
        
    Returns:
        float: The trend changes score.
    """
    y_df = pd.DataFrame([y_test, y_pred]).T
    y_df.columns = ["y_test", "y_pred"]
    y_df["y_test_shifted"] = y_df["y_test"].shift(-1)
    y_df["is_changed_trend_test"] = y_df["y_test"] != y_df["y_test_shifted"]
    y_df["y_predict_shifted"] = y_df["y_pred"].shift(-1)
    y_df["is_changed_trend_predict"] = y_df["y_pred"] != y_df["y_predict_shifted"]
    return classification_report(
        y_df["is_changed_trend_test"][:-1], 
        y_df["is_changed_trend_predict"][:-1], 
        digits=4
    )

def trend_changes_true(y_test: np.array, y_pred: np.array) -> float:
    """
    Calculate the trend changes score based on the test and predicted values.
    
    Args:
        y_test (np.array): True labels.
        y_pred (np.array): Predicted labels.
        
    Returns:
        float: The trend changes score.
    """
    y_df = pd.DataFrame([y_test, y_pred]).T
    y_df.columns = ["y_test", "y_pred"]
    y_df["y_test_shifted"] = y_df["y_test"].shift(-1)
    y_df["is_changed_trend_test"] = y_df["y_test"] != y_df["y_test_shifted"]
    y_df["y_predict_shifted"] = y_df["y_pred"].shift(-1)
    y_df["is_changed_trend_predict"] = y_df["y_pred"] != y_df["y_predict_shifted"]
    report = classification_report(
        y_df["is_changed_trend_test"][:-1],
        y_df["is_changed_trend_predict"][:-1],
        output_dict=True,
        zero_division=0
    )
    return report["True"][SCORE]

In [54]:
# Cargar datos
train = pd.read_csv("../../../data/post_cleaning/training_set.csv", parse_dates=["date"])
val = pd.read_csv("../../../data/post_cleaning/validation_set.csv", parse_dates=["date"])
X_train = train.drop(columns=["date", "target_trend"]).values
y_train = train["target_trend"].values
X_val = val.drop(columns=["date", "target_trend"]).values
y_val = val["target_trend"].values


In [55]:
# Mapear clases (-1,0,1) -> (0,1,2) para LightGBM
cls_map = {-1:0, 0:1, 1:2}
inv_map = {v:k for k,v in cls_map.items()}
y_train_m = np.vectorize(cls_map.get)(y_train)
y_val_m   = np.vectorize(cls_map.get)(y_val)

# Pesos de clase (inverso de la frecuencia)
from collections import Counter
cnt = Counter(y_train_m)
total = len(y_train_m)
class_weight = {c: total/(len(cnt)*n) for c,n in cnt.items()}

In [56]:
def objective(trial):
    set_seeds(SEED)  # Fijar semilla antes de cada trial
    # Hiperparámetros a optimizar
    param = {
        "objective": "multiclass",
        "num_class": 3,
        "learning_rate": trial.suggest_float("learning_rate", 0.03, 0.1, log=True), # 0.005, 0.2
        "num_leaves": trial.suggest_int("num_leaves", 5, 50), # 15, 100
        "max_depth": trial.suggest_int("max_depth", 2, 2), # 3, 12
        "feature_fraction": trial.suggest_float("feature_fraction", 0.9, 1), # 0.6, 1.0
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.94, 1), # 0.6, 1.0
        "bagging_freq": trial.suggest_int("bagging_freq", 8, 12), # 1, 10
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 20, 30), # 10, 100
        "metric": ["multi_logloss"],
        "verbosity": -1,
        "seed": SEED
    }

    # Dataset y pesos igual que antes
    lgb_train = lgb.Dataset(X_train, label=y_train_m, weight=[class_weight[c] for c in y_train_m])
    lgb_val = lgb.Dataset(X_val, label=y_val_m, weight=[class_weight[c] for c in y_val_m], reference=lgb_train)

    model = lgb.train(
        param,
        lgb_train,
        num_boost_round=1000,
        valid_sets=[lgb_val],
        valid_names=["val"],
        callbacks=[
            lgb.early_stopping(stopping_rounds=50, verbose=False)
        ]
    )

    # Predicción y métrica personalizada
    y_val_prob = model.predict(X_val, num_iteration=model.best_iteration)
    y_val_pred_m = y_val_prob.argmax(axis=1)
    score = trend_changes_true(y_val_m, y_val_pred_m)

    return score  # Optuna maximiza este valor

In [57]:
# Crear estudio y medir tiempo de Optuna
t0 = time.perf_counter()

study = optuna.create_study(
    direction="maximize",
    sampler=optuna.samplers.TPESampler(seed=SEED)
)
study.optimize(objective, n_trials=1500)
opt_duration_sec = time.perf_counter() - t0
n_trials_run = len(study.trials)

print(f"Optuna duró {opt_duration_sec:.2f}s en {n_trials_run} trials")


[I 2025-09-17 21:11:20,716] A new study created in memory with name: no-name-d560e02d-eb77-4625-bacd-af785b2164e0


[I 2025-09-17 21:11:20,991] Trial 0 finished with value: 0.546875 and parameters: {'learning_rate': 0.037780065334689, 'num_leaves': 33, 'max_depth': 2, 'feature_fraction': 0.9437727739007115, 'bagging_fraction': 0.9871215150228262, 'bagging_freq': 11, 'min_data_in_leaf': 22}. Best is trial 0 with value: 0.546875.
[I 2025-09-17 21:11:21,176] Trial 1 finished with value: 0.5303030303030303 and parameters: {'learning_rate': 0.04184836910744748, 'num_leaves': 41, 'max_depth': 2, 'feature_fraction': 0.9958139353683705, 'bagging_fraction': 0.9925559580845257, 'bagging_freq': 9, 'min_data_in_leaf': 25}. Best is trial 0 with value: 0.546875.
[I 2025-09-17 21:11:21,289] Trial 2 finished with value: 0.5230769230769231 and parameters: {'learning_rate': 0.06831081969967269, 'num_leaves': 37, 'max_depth': 2, 'feature_fraction': 0.9370250754790395, 'bagging_fraction': 0.9736717711639374, 'bagging_freq': 10, 'min_data_in_leaf': 20}. Best is trial 0 with value: 0.546875.
[I 2025-09-17 21:11:21,454] T

Optuna duró 358.20s en 1500 trials


In [58]:
# Visualizar el historial de optimización
plot_optimization_history(study)

In [59]:
print("Mejores hiperparámetros encontrados:")
print(study.best_params)
print(f"Mejor score de {SCORE}: {study.best_value:.4f}")
best_params = study.best_params

Mejores hiperparámetros encontrados:
{'learning_rate': 0.03457120366286536, 'num_leaves': 35, 'max_depth': 2, 'feature_fraction': 0.9577017858305965, 'bagging_fraction': 0.9563976601561844, 'bagging_freq': 9, 'min_data_in_leaf': 22}
Mejor score de f1-score: 0.6119


In [60]:
# --- Celda Nueva: Análisis de Hiperparámetros ---
from optuna.visualization import plot_param_importances, plot_slice

# 1. Gráfico de Importancia de Hiperparámetros
# Muestra qué hiperparámetros tuvieron el mayor impacto en el score.
param_importances = plot_param_importances(study)
param_importances.show()

# 2. Gráfico de Corte (Slice Plot)
# Muestra cómo varía el score para cada valor de cada hiperparámetro.
# Es excelente para ver los "rangos buenos" de cada parámetro.
slice_plot = plot_slice(study)
slice_plot.show()

In [61]:
# Guardar los mejores hiperparámetros y su valor
history = []
if os.path.exists("best_hyperparams.json"):
    try:
        with open("best_hyperparams.json", "r") as f:
            history = json.load(f)
    except (json.JSONDecodeError, ValueError):
        history = []

# Guardar ambos en un solo diccionario
history.append({
    "params": study.best_params,
    "value": study.best_value
})
# Guardar el historial de hiperparámetros
with open("best_hyperparams.json", "w") as f:
    json.dump(history, f, indent=2)

In [62]:
# Cargar historial de hiperparámetros y valores
import json

with open("best_hyperparams.json", "r") as f:
    history = json.load(f)

# Escoger el último (más reciente)
best_params = history[-1]["params"]
best_value = history[-1]["value"]

# Para ver todos:
for i, entry in enumerate(history):
    print(f"Hiperparámetros #{i+1}: {entry['params']}, Valor: {entry['value']}")

# Para escoger uno específico (por índice):
# best_params = history[indice_que_quieras]["params"]
# best_value = history[indice_que_quieras]["value"]

Hiperparámetros #1: {'learning_rate': 0.11399002785666924, 'num_leaves': 75, 'max_depth': 3, 'feature_fraction': 0.9262224100434759, 'bagging_fraction': 0.9308897359113296, 'bagging_freq': 10, 'min_data_in_leaf': 12}, Valor: 0.5942028985507246
Hiperparámetros #2: {'learning_rate': 0.03457120366286536, 'num_leaves': 35, 'max_depth': 2, 'feature_fraction': 0.9577017858305965, 'bagging_fraction': 0.9563976601561844, 'bagging_freq': 9, 'min_data_in_leaf': 22}, Valor: 0.6119402985074627


In [65]:
# Entrenar modelo final con los mejores hiperparámetros de Optuna
set_seeds(SEED)
final_params = best_params.copy()
final_params.update({
    "objective": "multiclass",
    "num_class": 3,
    "metric": ["multi_logloss"],  # Igual que en Optuna
    "verbosity": -1,
    "seed": SEED
})

# Re-crear los Dataset para evitar cualquier caché de pre-filtro
lgb_train = lgb.Dataset(X_train, label=y_train_m, weight=[class_weight[c] for c in y_train_m])
lgb_val   = lgb.Dataset(X_val,   label=y_val_m,   weight=[class_weight[c] for c in y_val_m], reference=lgb_train)

final_model = lgb.train(
    final_params,
    lgb_train,
    num_boost_round=1000,  # Igual que en Optuna
    valid_sets=[lgb_val],  # Igual que en Optuna
    valid_names=["val"],
    callbacks=[
        lgb.early_stopping(stopping_rounds=50, verbose=True)  # Igual que en Optuna
    ]
)


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[165]	val's multi_logloss: 0.543444


In [69]:
# Save model after training
final_model.save_model('../../../score_models/models/LightGBM.txt')

In [70]:
# Predicciones con el modelo final
y_val_prob = final_model.predict(X_val, num_iteration=final_model.best_iteration)
y_val_pred = y_val_prob.argmax(axis=1)
print("LightGBM FINAL report trend change score:\n",
      trend_changes_score(y_val_m, y_val_pred))

LightGBM FINAL report trend change score:
               precision    recall  f1-score   support

       False     0.8584    0.9132    0.8850       219
        True     0.6833    0.5541    0.6119        74

    accuracy                         0.8225       293
   macro avg     0.7709    0.7336    0.7484       293
weighted avg     0.8142    0.8225    0.8160       293



In [71]:
# Reporte completo: precisión, recall y F1 por clase
report = classification_report(y_val_m, y_val_pred, digits=4)
print("LightGBM Report:\n", report)
print("Balanced accuracy:", balanced_accuracy_score(y_val_m, y_val_pred))

LightGBM Report:
               precision    recall  f1-score   support

           0     0.7706    0.9130    0.8358        92
           1     0.2222    0.0816    0.1194        49
           2     0.8024    0.8758    0.8375       153

    accuracy                         0.7551       294
   macro avg     0.5984    0.6235    0.5976       294
weighted avg     0.6958    0.7551    0.7173       294

Balanced accuracy: 0.6234977082620488


In [72]:
# --- Exportar y Comparar Métricas de Modelos (Validación) ---

# 1. Definir el nombre del modelo actual y el archivo de salida
output_file = '../../../score_models/model_comparison_metrics.csv'

# 2. Calcular el reporte de clasificación estándar
# Usamos y_test_m y y_test_pred que están en la misma escala (0,1,2)
report_dict = classification_report(y_val_m, y_val_pred, output_dict=True, zero_division=0)
precision = report_dict['macro avg']['precision']
recall = report_dict['macro avg']['recall']
f1_score = report_dict['macro avg']['f1-score']


# 3. Calcular el reporte de cambio de tendencia
report = get_trend_changes_report_dict(y_val_m, y_val_pred)
trend_change_precision = report['True']['precision']
trend_change_recall = report['True']['recall']
trend_change_f1_score = report['True']['f1-score']

# 4. Organizar las nuevas métricas
new_metrics = {
    'precision': precision,
    'recall': recall,
    'f1_score': f1_score,
    'trend_change_precision': trend_change_precision,
    'trend_change_recall': trend_change_recall,
    'trend_change_f1_score': trend_change_f1_score,
    "optuna_duration_sec": opt_duration_sec,
    "n_trials": n_trials_run,
}

# 5. Cargar, actualizar y guardar el DataFrame de comparación
try:
    # Intentar cargar el archivo existente
    comparison_df = pd.read_csv(output_file, index_col='model')
    # Si existe, actualizar o añadir la fila para el modelo actual
    comparison_df.loc[model_name] = new_metrics
except FileNotFoundError:
    # Si no existe, crear un DataFrame nuevo directamente con los datos actuales
    comparison_df = pd.DataFrame([new_metrics], index=[model_name])

# Guardar el DataFrame actualizado en el CSV
comparison_df.to_csv(output_file, index_label='model')